## Análise comparativa de modelos

In [1]:
# Importando as bibliotecas necessárias
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, ShuffleSplit, GridSearchCV, cross_validate, train_test_split
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

import warnings

### 1. Obtenção de Dados

Nessa etapa obtemos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pré-processamento.

In [2]:
# Carregar seu dataset
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
display(df, df_dict)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


,variavel,descricao,tipo,subtipo
0,survived,Binário (0 ou 1) - Indica se o passageiro sobr...,qualitativa,nominal
1,pclass,Classe da passagem,qualitativa,ordinal
2,sex,Sexo dos passageiros,qualitativa,nominal
3,age,Idade,quantitativa,continua
4,sibsp,Quantidade de irmãos e cônjuges a bordo,quantitativa,discreta
5,parch,"Quantidade de filhos e enteados, pai ou mãe - ...",quantitativa,discreta
6,fare,Valor da passagem,quantitativa,continua
7,embarked,Portão de embarque,qualitativa,nominal
8,class,Classe da passagem,qualitativa,ordinal
9,who,"Categoriza o passageiro em homem, mulher e cri...",qualitativa,nominal


### 2. Preparação de dados

- Aqui organizamos os dados para a modelagem, separando a variável-alvo das variáveis independentes e categorizando cada variável conforme seu tipo "(nominal, ordinal, discreta, contínua)". Isso permite aplicar pré-processamentos específicos, como **codificação** para variáveis nominais e **normalização** para variáveis contínuas, garantindo que o modelo de machine learning receba dados limpos e prontos para análise.

#### Dados Faltantes

In [3]:
## Checando dados faltantes
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [4]:
print(f"Quantidade de instâncias: {df.shape[0]}")
print(f"Quantidade de variáveis: {df.shape[1]}")

Quantidade de instâncias: 891
Quantidade de variáveis: 15


In [5]:
# Remover as colunas indesejadas
columns_to_remove = ['deck']
df = df.drop(columns=columns_to_remove, axis=1)

# Atualizar o dicionário de variáveis se necessário
df_dict = df_dict[~df_dict['variavel'].isin(columns_to_remove)]

target_column = 'survived'

# Filtrando as colunas 
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)

ordinal_columns = (
    df_dict
    .query("subtipo == 'ordinais'and variavel != @target_column")
    .variavel
    .to_list()
)

discrete_columns = (
    df_dict
    .query("subtipo == 'discrete'")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [6]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


### 2.1 Pré-processamento

In [15]:
# 1. Pipeline para variáveis nominais
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Imputação pela moda
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # Codificação OneHotEncoder
   # ('normalization', StandardScaler())  # normalização
])

# 2. Pipeline para variáveis ordinais
ordinal_preprocessor = Pipeline([
    #('missing', SimpleImputer(strategy='most_frequent')),  # Imputação pela moda
    ('encoding', OrdinalEncoder()),  # Codificação OrdinalEncoder
    ('normalization', StandardScaler())
])

# 3. Pipeline para variáveis discretas
discrete_preprocessor = Pipeline([
    #('missing', SimpleImputer(strategy='most_frequent')),  # Imputação pela moda
    ('scaling', StandardScaler())  # Normalização
])

# 4. Pipeline para variáveis contínuas
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Imputação pela média
    ('scaling', StandardScaler())  # Normalização
])

# Combinação de todos os preprocessamentos em um único ColumnTransformer
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('ordinal', ordinal_preprocessor, ordinal_columns),
    ('discrete', discrete_preprocessor, discrete_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

### 3. Seleção de modelos
Iremos análisar quatro modelos, que serão testados utilizando um método de validação, a saber:

- **Logistic Regression (Regressão Logística):** Um modelo simples, porém eficaz, para classificação binária, especialmente se as variáveis preditoras tiverem relações lineares.
- **Random Forest (Floresta Aleatória):** Um modelo de ensemble que melhora a robustez e a generalização em comparação a uma única árvore de decisão.
- **Decision Tree (Árvore de Decisão):** O modelo usa uma estrutura em forma de árvore para tomar decisões ou fazer previsões com base em testes em atributos
- **K-Nearest Neighbors (KNN):** Modelo que classifica dados com base na proximidade dos pontos de dados mais próximos.
 
Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

- **Accuracy (acurácia):** Proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados;
- **Precision (precisão):** Proporção entre dados corretamente previstos como positivos e o total de observações positivas.
- **Recall:** proporção entre dados corretamente previstos como positivos com o total de observações.
- **F1-score:** Média entre precision e recall, portanto levando em conta tanto falsos positivos quanto falsos negativos.
- **ROC AUC:** Área sob a curva ROC, útil para avaliar a performance do modelo em diferentes limiares de classificação.

In [16]:
# Configurações para GridSearchCV e Cross-validation
n_folds_grid_search = 5
n_splits_comparative_analysis = 10
test_size = .2
random_state = 42
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Definindo os modelos e seus parâmetros
max_iter = 1000
models = [
    ('Logistic Regression', LogisticRegression(max_iter=max_iter, solver = 'liblinear'), {'C' : np.logspace(-4, 4, 20)}),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
    ('Decision Tree', DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance']}),   
]

In [17]:
warnings.filterwarnings("ignore")  # Ignora os avisos sobre as categorias desconhecidas

In [18]:
# Inicializando o DataFrame para armazenar os resultados
results = pd.DataFrame({})
# Inicializando os validadores de amostragem
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
# Iterar sobre os modelos para treinamento e avaliação
for model_name, model_object, model_parameters in models:
    print(f"Running {model_name}...")
    
    # Pipeline e GridSearchCV
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,  
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
      
    # Criando o pipeline com o preprocessor e o modelo (model_grid_search)
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    
    # Avaliação com cross_validate
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=-1,
        scoring=metrics
    )
    
    scores['model_name'] = [model_name] * n_splits_comparative_analysis
    display(pd.DataFrame(scores).select_dtypes(include=[float, int]).agg(['mean', 'std']))
    results = pd.concat([results, pd.DataFrame(scores)], ignore_index=True)

Running Logistic Regression...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,2.132071,0.088301,1.0,1.0,1.0,1.0
std,0.273574,0.027177,0.0,0.0,0.0,0.0


Running Random Forest...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,14.794390,0.092834,1.0,1.0,1.0,1.0
std,0.392731,0.040449,0.0,0.0,0.0,0.0


Running Decision Tree...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,0.664878,0.078790,1.0,1.0,1.0,1.0
std,0.072649,0.023314,0.0,0.0,0.0,0.0


Running K-Nearest Neighbors...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,5.601320,0.111767,0.983799,0.982115,0.983691,0.982836
std,0.859618,0.075744,0.011614,0.013055,0.011907,0.012445


In [19]:
# Função para destacar o melhor desempenho
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

# Exibição dos resultados finais
display(Markdown("### 3.1 Resultados gerais"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 3.1 Resultados gerais

- Modelos Perfeitos (Overfitting): Os resultados para Decision Tree, Logistic Regression, e Random Forest são perfeitos em termos de acurácia, precisão, recall, e F1. No entanto, isso é um forte indicativo de overfitting, onde o modelo se ajustou tão bem aos dados de treino que pode estar falhando em generalizar para dados novos.

- `K-Nearest Neighbors` é o modelo que pode ser mais confiável devido ao seu desempenho ligeiramente inferior, o que sugere que ele está generalizando melhor para os dados de teste. Menos propenso ao overfitting, embora ainda seja muito eficaz.

### 3.2 Persistência do modelo

In [20]:
#Obtem o modelo e os parametros ganhadores
model_name, model_object, model_parameters  = [foo for foo in models if foo[0] == "Decision Tree"][0] 


model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )

approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model_grid_search)
])

approach.fit(X, y) #Seleciona o approach

print(f"Hiper parâmetros do modelo: {approach.steps[1][1].best_params_}")

Hiper parâmetros do modelo: {'criterion': 'gini', 'max_depth': 3}


In [21]:
joblib.dump(approach, '../models/model.joblib') # Salva o modelo em disco

['../models/model.joblib']